In [73]:
 pip install langchain pinecone-client openai langchain_openai langchain_pinecone

Note: you may need to restart the kernel to use updated packages.


In [74]:
from dotenv import load_dotenv
import os

# Load environment variables from the .env files
load_dotenv("key.env")  # For OpenAI API key
load_dotenv("pineconekey.env")  # For Pinecone API key
load_dotenv("langkey.env")
# Get the OpenAI API key from the environment
openai_api_key = os.getenv("OPENAI_API_KEY")

# Get the Pinecone API key from the environment
pinecone_api_key = os.getenv("PINECONE_API_KEY")
# 
langsmith_api_key=os.getenv("LANGSMITH_API_KEY")

# Check if the keys were loaded correctly
if openai_api_key:
    print("OpenAI API Key loaded successfully!")
else:
    print("OpenAI API Key not found!")

if pinecone_api_key:
    print("Pinecone API Key loaded successfully!")
else:
    print("Pinecone API Key not found!")

if langsmith_api_key:
     print("lang API Key loaded successfully!")
else:
    print("lang API Key not found!")

OpenAI API Key loaded successfully!
Pinecone API Key loaded successfully!
lang API Key loaded successfully!


In [75]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from pinecone import Pinecone
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub

In [76]:
import os

# Check if the index exists
index_name = "sample-movies"
if index_name not in pc.list_indexes().names():
    # Create the index if it doesn't exist
    pc.create_index(
        name=index_name,
        dimension=1024,  # Replace with your embedding dimensions
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
print(f"Index '{index_name}' created successfully!")


Index 'sample-movies' created successfully!


In [89]:
pc = Pinecone()

index_name = 'sample-movies'
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [91]:
llm = ChatOpenAI(model_name="gpt-4", temperature=0.0)

text_field = "text"
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')
vectorstore = PineconeVectorStore(index, embeddings, text_field)

In [93]:
# Define whole chain
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})


In [95]:
# The function to combine multiple document into one
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [99]:
# Get a prompt from LangChain hub
prompt = hub.pull("rlm/rag-prompt")

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("I want to watch a movie about adventure?"):
    print(chunk, end="", flush=True)

Your question seems to be missing specific context. However, there are many adventure movies you might enjoy such as "Indiana Jones", "Pirates of the Caribbean", or "The Lord of the Rings" series.